# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)
output_data_file = "output_data/cities.csv"
weather_df = pd.read_csv(output_data_file)
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Beloha,98,MG,1570871370,42,-25.17,45.06,85.69,8.71
1,Payo,100,ID,1570871370,37,-3.75,103.64,86.61,5.90
2,New Norfolk,20,AU,1570871370,57,-42.78,147.06,50.00,9.17
3,Port Elizabeth,1,US,1570871371,87,39.31,-74.98,57.99,3.36
4,Ternate,20,PH,1570871371,70,14.29,120.72,88.00,11.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)

# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
#                                  , 
#                                  dissipating=False, max_intensity=10,
#                                  point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
weather_df.head()
weather_df = weather_df[(weather_df['Max Temp'] < 80) & (weather_df['Max Temp'] > 70) 
                        & (weather_df['Wind Speed'] < 10) & (weather_df['Cloudiness'] == 0)]
weather_df.head(10)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
53,Bluff,0,AU,1570871389,21,-23.58,149.07,72.93,2.47
156,Guardiagrele,0,IT,1570871419,82,42.20,14.22,71.01,3.36
168,Samarai,0,PG,1570871423,87,-10.62,150.67,71.73,3.68
198,Daru,0,PG,1570871431,93,-9.07,143.21,73.58,5.39
226,Porto Murtinho,0,BR,1570871439,74,-21.70,-57.88,76.15,7.85
246,Dukat,0,AL,1570871445,63,40.25,19.57,79.00,3.00
262,Rafsanjan,0,IR,1570871449,12,30.40,55.99,78.28,6.95
271,Ingham,0,AU,1570871452,78,-18.65,146.16,70.86,2.44
299,Camocim,0,BR,1570871460,84,-2.90,-40.84,72.19,5.80
393,Yeppoon,0,AU,1570871494,60,-23.13,150.74,73.40,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = weather_df.copy()
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
53,Bluff,0,AU,1570871389,21,-23.58,149.07,72.93,2.47
156,Guardiagrele,0,IT,1570871419,82,42.20,14.22,71.01,3.36
168,Samarai,0,PG,1570871423,87,-10.62,150.67,71.73,3.68
198,Daru,0,PG,1570871431,93,-9.07,143.21,73.58,5.39
226,Porto Murtinho,0,BR,1570871439,74,-21.70,-57.88,76.15,7.85


In [6]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
53,Bluff,0,AU,1570871389,21,-23.58,149.07,72.93,2.47,
156,Guardiagrele,0,IT,1570871419,82,42.20,14.22,71.01,3.36,
168,Samarai,0,PG,1570871423,87,-10.62,150.67,71.73,3.68,
198,Daru,0,PG,1570871431,93,-9.07,143.21,73.58,5.39,
226,Porto Murtinho,0,BR,1570871439,74,-21.70,-57.88,76.15,7.85,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
         hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [8]:
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
53,Bluff,0,AU,1570871389,21,-23.58,149.07,72.93,2.47,Bluff Hotel
156,Guardiagrele,0,IT,1570871419,82,42.20,14.22,71.01,3.36,Hotel Sole
168,Samarai,0,PG,1570871423,87,-10.62,150.67,71.73,3.68,Tawali Resort
198,Daru,0,PG,1570871431,93,-9.07,143.21,73.58,5.39,Daru hotel
226,Porto Murtinho,0,BR,1570871439,74,-21.70,-57.88,76.15,7.85,Hotel dos Camalotes


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel location: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…